# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Import the Output File (CSV) from the WeatherPy previous activity
input_df = pd.read_csv("../output_data/weather_df.csv")
input_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Hithadhoo,-0.6000,73.0833,82.99,68,5,7.18,MV,1619312286
1,Nuuk,64.1835,-51.7216,39.20,65,20,17.27,GL,1619312273
2,Busselton,-33.6500,115.3333,66.65,75,0,0.85,AU,1619312336
3,Manta,-0.9500,-80.7333,77.00,83,40,6.91,EC,1619312187
4,Atuona,-9.8000,-139.0333,80.76,72,90,15.64,PF,1619312337


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
# Store latitude and longitude in locations to use as a heatmap parameter
locations = input_df[["Lat","Lng"]]
locations.reset_index
locations.head()

,Lat,Lng
0,-0.6000,73.0833
1,64.1835,-51.7216
2,-33.6500,115.3333
3,-0.9500,-80.7333
4,-9.8000,-139.0333


In [5]:
# Confirm Humidity is an integer
input_df["Humidity"].dtype

# Find highest humidity for the max intensity

max_humidity = input_df["Humidity"].max()
print(max_humidity)

# Store humidity series as a variable to use as a heatmap parameter
humidity = input_df["Humidity"]

100


In [6]:
# Print the gmap
fig = gmaps.figure()
#fig

# Create the heatmap layer
heat_layer = gmaps.heatmap_layer(locations,
                                 weights=humidity,
                                 dissipating=False,
                                 max_intensity=100,
                                 point_radius=3                                
                                )
# Add heatmap layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
# Get ideal weather condition rows in a new DF
ideal_weather_df = input_df.loc[(input_df["Max Temp"] > 65) & (input_df["Max Temp"] < 82) 
                            & (input_df.Humidity > 20) & (input_df.Humidity < 50)
                            & (input_df.Cloudiness < 10) & (input_df["Wind Speed"] <15)
                           ,:]
ideal_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
72,Koungheul,13.9833,-14.8000,78.87,31,2,10.80,SN,1619312362
125,Alice Springs,-23.7000,133.8833,66.20,39,0,10.36,AU,1619312249
151,Cabo San Lucas,22.8909,-109.9124,79.00,27,1,9.22,MX,1619312387
190,Altus,34.6381,-99.3340,69.80,35,1,4.61,US,1619312401
598,Tharād,24.3961,71.6267,79.86,33,0,4.79,IN,1619312550
614,Jollyville,30.4427,-97.7750,78.80,38,1,6.91,US,1619312438


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
# Define base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        
# Define the keyword
keyword = "hotel"

# Define the type
place_type = "lodging"

# Define radius
radius = 5000

# Hotel list

hotels_list = []

# Iterate through the ideal_weather_df

for index, row in ideal_weather_df.iterrows():
    
    # Grab lat and long for each row in the DF
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Assemble url and make API request
    response = requests.get(f"""
                            {base_url}?location={lat},{lng}
                            &radius={radius}
                            &keyword={keyword}
                            &type={place_type}
                            &key={g_key}""").json()
    # View json of first hotel from each row of the DF                       
    #pprint(response["results"][0]["name"])
    # Parse json to find hotel
    hotels_list.append(response["results"][0]["name"])
print(hotels_list)

['BOA Agence Nguekhokh', 'DoubleTree by Hilton Hotel Alice Springs', 'Bahia Hotel & Beach House', 'Holiday Inn Express & Suites Altus', 'HOTEL DESERT INN THARAD', 'The Westin Austin at The Domain']


In [9]:
# Add the hotels to the DF
ideal_weather_df["Hotel Name"] = hotels_list
hotel_df = ideal_weather_df

# Clean up the DF - drop columns we don't need anymore
hotel_df = hotel_df.drop(columns=["Max Temp", "Humidity", "Cloudiness", "Wind Speed", "Date"])

# Clean up the DF - re-order the columns
hotel_df = hotel_df[["City", "Country", "Lat", "Lng", "Hotel Name"]]

# Print the clean DF
hotel_df

<ipython-input-9-4cc927067276>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ideal_weather_df["Hotel Name"] = hotels_list


,City,Country,Lat,Lng,Hotel Name
72,Koungheul,SN,13.9833,-14.8000,BOA Agence Nguekhokh
125,Alice Springs,AU,-23.7000,133.8833,DoubleTree by Hilton Hotel Alice Springs
151,Cabo San Lucas,MX,22.8909,-109.9124,Bahia Hotel & Beach House
190,Altus,US,34.6381,-99.3340,Holiday Inn Express & Suites Altus
598,Tharād,IN,24.3961,71.6267,HOTEL DESERT INN THARAD
614,Jollyville,US,30.4427,-97.7750,The Westin Austin at The Domain


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer on top of heat map
fig = gmaps.figure()

marker_layer = gmaps.marker_layer(locations,
                                  info_box_content=hotel_info)
# Display figure
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))